In [1]:
import pandas as pd
import numpy as np
import time
from time import mktime
from datetime import datetime, timedelta
import geopy.distance
import gc
import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

import data_prep_functions
import request_api_nba_stats

### BASES PARA PREDICT

In [2]:
all_games = pd.read_csv("all_games_nba_18_19_11-11-2018.csv").drop(["Unnamed: 0"], axis=1)

#### Fluxo Atualiza Rodada de Ontem

In [43]:
# AMANHÃ = datetime.fromtimestamp(mktime(time.localtime(time.time() + 24*3600)))
# HOJE = datetime.today()

In [6]:
games_ids = request_api_nba_stats.get_list_gameids(datetime.today() - timedelta(1), start_at=len(all_games)) 

In [7]:
df_full, df_full_jogo, erros = request_api_nba_stats.get_nba_stats_data(games_ids, ['traditional', 'advanced', 'scoring', 'misc', 'usage', 'fourfactors', 'playertrack', 'defensive'])

In [8]:
new_base_nba_2018 = pd.concat(df_full_jogo).reset_index()

In [9]:
new_all_games = data_prep_functions.prepara_base(new_base_nba_2018).reset_index()
new_all_games = data_prep_functions.cria_features(new_all_games)

In [10]:
#all_games = new_all_games
all_games = pd.concat([all_games, new_all_games]).reset_index(drop=True)

In [12]:
all_games["DATE"] = [datetime.strptime(str(x)[:10], '%Y-%m-%d') for x in all_games["DATE"]]

In [18]:
#resp = data_prep_functions.gera_last_N_games(all_games, N=[5])
delta_L5_2018 = data_prep_functions.variaveis_delta(resp, to_predict=False, keep_features=['team_home', 'team_away', 'DATE', 'DISTANCE_KM_home', 
                                                                                           'DISTANCE_KM_away', 'DAYS_FROM_LAST_GAME_home', 'DAYS_FROM_LAST_GAME_away',
                                                                                           'fl_home_win', 'team_home_game_num', 'team_away_game_num',
                                                                                           "PTS_home", "PTS_away"])
#delta_L5_2018.to_csv("delta_cross_L5_nba_18_19.csv")

In [45]:
delta_L5_2018 = delta_L5_2018.loc[:, ~delta_L5_2018.columns.duplicated()]

In [50]:
to_keep = ["N_GAMES_L2_days", "E_DEF_RATING", "DEF_RATING", "OPP_PTS_2ND_CHANCE", "N_GAMES_L6_days",
"OPP_PTS_OFF_TOV", "BLKA", "E_NET_RATING", "NET_RATING", "PLUS_MINUS", "PIE", "NET_RATING",
"N_WINS_TOTAL", "WIN_PCT", "WIN_AWAY_PCT", "STD_DAYS_FROM_LG", "WIN_HOME_PCT", "PTS_PAINT",
"N_WINS_HOME", "PTS_FB", "DREB", "FG_PCT", "REB", "OFF_RATING"]

In [51]:
def contains(name, lista):
    for el in lista:
        if el in name:
            return True
    return False

In [53]:
delta_L5_2018[['team_home',
 'team_away',
 'DATE',
 'DISTANCE_KM_home',
 'DISTANCE_KM_away',
 'DAYS_FROM_LAST_GAME_home',
 'DAYS_FROM_LAST_GAME_away',
 'fl_home_win',
 'team_home_game_num',
 'team_away_game_num',
 'PTS_home',
 'PTS_away',
 'fl_home_win',
 'team_home_game_num',
 'team_away_game_num'] + [x for x in delta_L5_2018.columns if contains(x, to_keep)]].to_csv("base_simples_2018.csv", sep=";", decimal=',', float_format='%.5f')

In [19]:
all_games.set_index("GAME").to_csv("all_games_nba_18_19_" + datetime.today().strftime('%d-%m-%Y') + ".csv")

In [20]:
all_games.set_index("GAME")

,AST_PCT_away,AST_PCT_home,AST_RATIO_away,AST_RATIO_home,AST_TOV_away,AST_TOV_home,AST_away,AST_home,AST_playertrack_away,AST_playertrack_home,...,USG_PCT_away,USG_PCT_home,USG_PCT_usage_away,USG_PCT_usage_home,fl_home_win,fl_playoff,team_away,team_away_game_num,team_home,team_home_game_num
GAME,,,,,,,,,,,,,,,,,,,,,
PHI @ BOS 2018-10-16,0.529,0.500,13.7,15.1,1.13,1.40,18,21,18.0,21.0,...,1.0,1.0,1.0,1.0,1,0,PHI,1,BOS,1
OKC @ GSW 2018-10-16,0.636,0.667,14.7,18.4,1.40,1.33,21,28,21.0,28.0,...,1.0,1.0,1.0,1.0,1,0,OKC,1,GSW,1
MIL @ CHA 2018-10-17,0.619,0.512,18.5,15.7,1.24,1.91,26,21,26.0,21.0,...,1.0,1.0,1.0,1.0,0,0,MIL,1,CHA,1
BKN @ DET 2018-10-17,0.700,0.538,20.2,15.0,1.47,1.24,28,21,21.0,28.0,...,1.0,1.0,1.0,1.0,1,0,BKN,1,DET,1
MEM @ IND 2018-10-17,0.640,0.617,13.1,21.1,1.60,1.45,16,29,16.0,29.0,...,1.0,1.0,1.0,1.0,1,0,MEM,1,IND,1
MIA @ ORL 2018-10-17,0.421,0.513,11.5,14.3,0.94,1.67,16,20,20.0,16.0,...,1.0,1.0,1.0,1.0,1,0,MIA,1,ORL,1
ATL @ NYK 2018-10-17,0.463,0.467,13.5,14.0,0.79,1.31,19,21,21.0,19.0,...,1.0,1.0,1.0,1.0,1,0,ATL,1,NYK,1
CLE @ TOR 2018-10-17,0.500,0.467,12.6,15.9,1.06,2.10,17,21,21.0,17.0,...,1.0,1.0,1.0,1.0,1,0,CLE,1,TOR,1
NOP @ HOU 2018-10-17,0.692,0.538,23.1,15.5,3.00,1.75,36,21,36.0,21.0,...,1.0,1.0,1.0,1.0,0,0,NOP,1,HOU,1


In [90]:
teste = data_prep_functions.gera_last_N_games(all_games, all_games.drop(['FATIGUE'],axis=1, errors="ignore"))

In [91]:
teste2 = data_prep_functions.variaveis_delta(teste, to_predict = False, keep_features=['team_home', 'team_away', 'DATE', 'DISTANCE_KM_home', 'DISTANCE_KM_away',
                                                                                       'DAYS_FROM_LAST_GAME_home', 'DAYS_FROM_LAST_GAME_away', "PTS_home", "PTS_away"])

In [94]:
teste2["D1_PTS"] = teste2["PTS_home"] - teste2["PTS_away"] 
teste2["FATIGUE"] = teste2["DAYS_FROM_LAST_GAME_home"] - teste2["DAYS_FROM_LAST_GAME_away"] 

In [103]:
teste2.to_csv("../base_2019_pts.csv", sep=";")

### Fluxo Predição no Dia

In [75]:
new_games_ids = request_api_nba_stats.get_list_gameids(max_date = datetime.fromtimestamp(mktime(time.localtime(time.time() + 24*3600)))
                                                   , year = '2018', start_at=len(all_games))

In [76]:
new_games = request_api_nba_stats.get_game_infos_dates(new_games_ids)

In [77]:
new_games = data_prep_functions.cria_features(new_games, all_games)

In [78]:
to_predict = data_prep_functions.gera_last_N_games(new_games, all_games.drop(['FATIGUE'],axis=1, errors="ignore"))

In [79]:
to_predict = data_prep_functions.variaveis_delta(to_predict)

In [80]:
to_predict["FL_HOME"] = np.where((to_predict['D2_PLUS_MINUS_L5'] > 16) | (to_predict['D1_WIN_PCT_L5'] > 0.6), 1, 0)
to_predict["FL_AWAY"] = np.where((to_predict['D2_PLUS_MINUS_L5'] < -8) | (to_predict['D1_WIN_PCT_L5'] < -0.6), 1, 0)
to_predict["FATIGUE"] = to_predict["DAYS_FROM_LAST_GAME_home"] - to_predict["DAYS_FROM_LAST_GAME_away"]

In [81]:
to_predict[['D2_PLUS_MINUS_L5', 'D1_WIN_PCT_L5', 'D1_NET_RATING_L5', "FATIGUE", 'FL_HOME', 'FL_AWAY']]

,D2_PLUS_MINUS_L5,D1_WIN_PCT_L5,D1_NET_RATING_L5,FATIGUE,FL_HOME,FL_AWAY
ORL @ LAL 2018-11-25,0.8,0.0,-0.92,0,0,0
PHX @ DET 2018-11-25,5.2,0.0,0.86,0,0,0
CHA @ ATL 2018-11-25,-5.4,-0.6,-12.92,0,0,0
PHI @ BKN 2018-11-25,-4.4,-0.4,-3.02,0,0,0
MIA @ TOR 2018-11-25,2.0,0.2,14.82,0,0,0
NYK @ MEM 2018-11-25,14.0,0.6,8.02,0,1,0
UTA @ SAC 2018-11-25,14.8,0.2,4.24,-1,0,0
LAC @ POR 2018-11-25,7.0,-0.2,-12.74,0,0,0
MIL @ CHA 2018-11-26,10.8,-0.2,-14.98,1,0,0
MIN @ CLE 2018-11-26,9.4,-0.2,-11.16,0,0,0
